In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification


In [2]:
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

In [7]:
from transformers import pipeline

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
nlp = nlp("Apple est créée le 1er avril 1976 dans le garage de la maison d'enfance de Steve Jobs à Los Altos en Californie par Steve Jobs retiré le 9 janvier 2015.")
nlp

[{'entity_group': 'ORG',
  'score': 0.99379635,
  'word': 'Apple',
  'start': 0,
  'end': 5},
 {'entity_group': 'PER',
  'score': 0.99533767,
  'word': 'Steve Jobs',
  'start': 74,
  'end': 85},
 {'entity_group': 'LOC',
  'score': 0.99801445,
  'word': 'Los Altos',
  'start': 87,
  'end': 97},
 {'entity_group': 'LOC',
  'score': 0.9980428,
  'word': 'Californie',
  'start': 100,
  'end': 111},
 {'entity_group': 'PER',
  'score': 0.9947664,
  'word': 'Steve Jobs',
  'start': 115,
  'end': 126}]

In [14]:
import pandas as pd
df = pd.DataFrame(nlp)
df = df.rename(columns={'entity_group': 'entity'})
count_df = df.groupby('entity').count().reset_index()[['entity', 'score']]
count_df = count_df.rename(columns={'score': 'count'})

  entity     score        word  start  end
0    ORG  0.993796       Apple      0    5
1    PER  0.995338  Steve Jobs     74   85
2    LOC  0.998014   Los Altos     87   97
3    LOC  0.998043  Californie    100  111
4    PER  0.994766  Steve Jobs    115  126


In [15]:
    from transformers import AutoTokenizer, AutoModelForTokenClassification
    tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
    model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")
    from transformers import pipeline

    nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
    nlp_camem = (nlp(input_text))
    df = pd.DataFrame(nlp_camem)
    df = df.rename(columns={'entity_group': 'entity'})
    count_df = df.groupby('entity').count().reset_index()[['entity', 'score']]
    count_df = count_df.rename(columns={'score': 'count'})
    st.dataframe(count_df)

In [16]:
count_df

,entity,count
0,LOC,2
1,ORG,1
2,PER,2


In [3]:
import urllib.request
import json
import os
import ssl
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

def ar_hugging_face(input_text):
    def allowSelfSignedHttps(allowed):
        # bypass the server certificate verification on client side
        if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
            ssl._create_default_https_context = ssl._create_unverified_context

    allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

    # Request data goes here
    # The example below assumes JSON formatting which may be updated
    # depending on the format your endpoint expects.
    # More information can be found here:
    # https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
    data = {"inputs" : input_text}

    body = str.encode(json.dumps(data))

    url = 'https://arabic-ner-srpel9nm.westeurope.inference.ml.azure.com/score'
    # Replace this with the primary/secondary key or AMLToken for the endpoint
    api_key = os.getenv('api_key')
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")

    # The azureml-model-deployment header will force the request to go to a specific deployment.
    # Remove this header to have the request observe the endpoint traffic rules
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'main' }

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        # print(result)
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        # print(error.info())
        # print(error.read().decode("utf8", 'ignore'))

    json_result = json.loads(result.decode('utf-8'))
    df = pd.DataFrame.from_dict(json_result)
    df['entity'] = df['entity'].apply(lambda x: x.split('-')[1])
    groupby_entity_counts = df.groupby('entity').size().reset_index(name='count')
    return df, groupby_entity_counts


In [7]:
df, groupby_entity_counts = ar_hugging_face("""
                                            أنا اسمي إلياس وأسكن في مدينة ليل في فرنسا.
                                            """)

In [8]:

groupby_entity_counts

,entity,count
0,LOCATION,2
1,PERSON,1


In [9]:
df

,entity,score,index,word,start,end
0,PERSON,0.991181,3,الياس,54,59
1,LOCATION,0.990755,8,ليل,75,78
2,LOCATION,0.997875,10,فرنسا,82,87
